# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'predict_diabetes'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-236265
Azure region: westeurope
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-236265


In [3]:
cluster_name = "mimi-compute"

try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {'--C': uniform(0.05, 0.5),
    '--max_iter': choice(5, 10, 50, 100)
    } )

#TODO: Create your estimator and hyperdrive config
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies (1).yml')

src = ScriptRunConfig(source_directory='.',
                      script='train (2).py',
                      compute_target=cluster_name,
                      environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=25,
                                     max_concurrent_runs=5)

In [5]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_56e47875-c5c3-4f1c-82a7-2b0b04341700
Web View: https://ml.azure.com/runs/HD_56e47875-c5c3-4f1c-82a7-2b0b04341700?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-236265/workspaces/quick-starts-ws-236265&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-06-21T19:04:13.268938][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space
[2023-06-21T19:04:13.7926153Z][SCHEDULER][INFO]Scheduling job, id='HD_56e47875-c5c3-4f1c-82a7-2b0b04341700_0' 
[2023-06-21T19:04:13.9276969Z][SCHEDULER][INFO]Scheduling job, id='HD_56e47875-c5c3-4f1c-82a7-2b0b04341700_1' 
[2023-06-21T19:04:14.1118528Z][SCHEDULER][INFO]Scheduling job, id='HD_56e47875-c5c3-4f1c-82a7-2b0b04341700_2' 
[2023-06-21T19:04:14.1119618Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_56e47875-c5c3-4f1c-82a7-2b0b04341700_0' 
[2023-06-21T19:04:14.1885104Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_56e47875-c5c3-4f1c-82a7-

{'runId': 'HD_56e47875-c5c3-4f1c-82a7-2b0b04341700',
 'target': 'mimi-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-06-21T19:04:12.585699Z',
 'endTimeUtc': '2023-06-21T19:19:14.833149Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'c57ed38c-d064-424d-8c7f-66410b870e3f',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': 'infinite_space_size',
  'score': '1.0',
  'best_child_run_id': 'HD_56e47875-c5c3-4f1c-82a7-2b0b04341700_7',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_56e47875-c5c3-4f1c-82a7-2b0b04341700_7'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClie

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [7]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print(best_run)
print('Accuracy:', best_run_metrics['Accuracy'])
print(f'Metrics: {best_run.get_metrics()}')

Run(Experiment: predict_diabetes,
Id: HD_56e47875-c5c3-4f1c-82a7-2b0b04341700_7,
Type: azureml.scriptrun,
Status: Completed)
Accuracy: 1.0
Metrics: {'Regularization Strength:': 0.34383145227349843, 'Max iterations:': 100, 'Accuracy': 1.0}


In [8]:
print(best_run.get_file_names())

['outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [12]:
best_run.download_file(best_run.get_file_names()[0], output_file_path='./outputs/')

In [13]:
joblib.load('./outputs/model.joblib')

Trying to unpickle estimator LogisticRegression from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.


LogisticRegression(C=0.34383145227349843, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
best_run.register_model(model_name='best-run',
                                      model_path='outputs/model.joblib',
                                      description='Best model from Hyperdrive for diabetes')

Model(workspace=Workspace.create(name='quick-starts-ws-236265', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-236265'), name=best-run, id=best-run:2, version=2, tags={}, properties={})

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

